PROCESS DOCUMENT

In [6]:
from IPython.display import Markdown
from utils.document_processor import DocumentProcessor
import os

docs = DocumentProcessor()

DATA_PATH = "./data"
extracted_docs = []
for filename in os.listdir(DATA_PATH):
  filepath = os.path.join(DATA_PATH, filename)
  with open(filepath, "rb") as f:
    document = f.read()
    result = docs.process_document(document, filename)
    extracted_docs.append(result)
    
content, images, tables, plain_text = extracted_docs[0] #1st file


TEST OLLAMA MODEL

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [8]:
OLLAMA_MODEL = "llama3.2"
COLLECTION_NAME = "ollama_vectore_test"

In [9]:
template = """
  You are a helpful assistant for text summarization. 
  Only include information that is part of the document. 
  Do not include your own opinion or analysis.

  Document: 
  "{document}"
  Summary:
"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model=OLLAMA_MODEL)

chain = prompt | model

response = chain.invoke({"document": plain_text})
Markdown(response)


Here is a summary of the document:

The person is Muhammad Fajrul Haqqi, a software developer with a Master's degree in Artificial Intelligence. He has experience as a full stack developer, chief technology officer, and data scientist at various companies and institutions.

He has worked on several projects, including developing a frontend technology standard for HIS 2.0 using NextJS, building the tech infrastructure standard for Fellas.id, and creating a cloud-based AI system to predict petrophysical data from fields.

He has won several awards, including the Riau Government Scholarship, Digital Forum 2022, and DELFI Data Science Hackathon. He has also presented his research at international conferences and events, such as the International Energy Summit (IES) and the Digital Forum 2022.

Some of his technical skills include Django REST Framework, React JS, Next JS, FAST API, Cloud SQL, Machine Learning, Deep Learning, Google App Engine, and Google Kubernetes Engine. He is fluent in English (B2 Upper-Intermediate) and has native proficiency in Indonesian.

TEST OPENAI MODEL

In [11]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

model = "gpt-35-turbo"
response = openai.ChatCompletion.create(
  engine=model,
  messages=[
    {
      "role": "system", 
      "content": """
        You are a helpful assistant for text summarization.
        Only include information that is part of the document. 
        Do not include your own opinion or analysis.
      """
    },
    {
      "role": "user",
      "content": plain_text
    }
  ],
)

Markdown(response.choices[0].message.content)



The document is a resume belonging to Muhammad Fajrul Haqqi, a software developer from Indonesia. Fajrul has a Master's degree in Artificial Intelligence and has worked in various job positions, such as full-stack developer, CTO, software developer, tech intern, and data science intern. He has had a project management role and taken responsibility to create technology standards in his previous workplaces. Fajrul has won various awards and has participated and won competitions both nationally and internationally. Additionally, he has received several scholarships, including the Riau Government Scholarship for his Bachelor and Master's degrees. As for technical skills, Fajrul has experience working with various technologies such as Django REST Framework, React JS, Next JS, FAST API, Cloud SQL, Data Analyst, Machine Learning, and Deep Learning.